Imports


In [1]:
import torch
import torchvision
import pandas as pd
import torch.nn as nn
from tqdm import tqdm
import multiprocessing
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import tabulate

Versión torch


In [2]:
print("Torch version: ", torch.__version__)

####################################################################
# Set Device
####################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

Torch version:  2.5.1+cu121
Device:  cuda


Preparar datos


In [3]:
####################################################################
# Prepare Data
####################################################################

train_set = torchvision.datasets.MNIST(".data/", train=True, download=True)
# train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)

test_set = torchvision.datasets.MNIST(".data/", train=False, download=True)
# test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True)

print("Train images: ", train_set)
print("Image: ", train_set[0][0])
print("Label: ", train_set[0][1])
print("Label one hot: ", F.one_hot(torch.tensor(train_set[0][1]), num_classes=10))

Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 9.91M/9.91M [00:00<00:00, 17.1MB/s]


Extracting .data/MNIST/raw/train-images-idx3-ubyte.gz to .data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 28.9k/28.9k [00:00<00:00, 496kB/s]


Extracting .data/MNIST/raw/train-labels-idx1-ubyte.gz to .data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.37MB/s]


Extracting .data/MNIST/raw/t10k-images-idx3-ubyte.gz to .data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 110] Connection timed out>



100%|██████████| 4.54k/4.54k [00:00<00:00, 2.92MB/s]


Extracting .data/MNIST/raw/t10k-labels-idx1-ubyte.gz to .data/MNIST/raw

Train images:  Dataset MNIST
    Number of datapoints: 60000
    Root location: .data/
    Split: Train
Image:  <PIL.Image.Image image mode=L size=28x28 at 0x7D781B89C990>
Label:  5
Label one hot:  tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])


Dataset class


In [4]:
####################################################################
# Dataset Class
####################################################################
class MNIST_dataset(Dataset):
    def __init__(self, data, partition="train"):
        print("\nLoading MNIST ", partition, " Dataset...")
        self.data = data
        self.partition = partition
        print("\tTotal Len.: ", len(self.data), "\n", 50 * "-")

    def __len__(self):
        return len(self.data)

    def from_pil_to_tensor(self, image):
        return torchvision.transforms.ToTensor()(image)

    def __getitem__(self, idx):
        # Image
        image = self.data[idx][0]
        # PIL Image to torch tensor
        image_tensor = self.from_pil_to_tensor(image)
        # care! net expect a 784 size vector and our dataset
        # provide 1x28x28 (channels, height, width) -> Reshape!
        image_tensor = image_tensor.view(-1)

        # Label
        label = torch.tensor(self.data[idx][1])
        label = F.one_hot(label, num_classes=10).float()

        return {"img": image_tensor, "label": label}


train_set = MNIST_dataset(train_set, partition="train")
test_set = MNIST_dataset(test_set, partition="test")


Loading MNIST  train  Dataset...
	Total Len.:  60000 
 --------------------------------------------------

Loading MNIST  test  Dataset...
	Total Len.:  10000 
 --------------------------------------------------


DataLoader class


In [5]:
####################################################################
# DataLoader Class
####################################################################

batch_size = 100
num_workers = multiprocessing.cpu_count() - 1
print("Num workers", num_workers)
train_dataloader = DataLoader(
    train_set, batch_size, shuffle=True, num_workers=num_workers
)
test_dataloader = DataLoader(
    test_set, batch_size, shuffle=False, num_workers=num_workers
)

Num workers 1


NN Class


In [6]:
####################################################################
# Neural Network Class
####################################################################
# Creating our Neural Network - Fully Connected
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(784, 512)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(512, 512)
        self.relu2 = nn.ReLU()
        self.linear3 = nn.Linear(512, 512)
        self.relu3 = nn.ReLU()
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        out = self.relu1(self.linear1(x))
        out = self.relu2(self.linear2(out))
        out = self.relu3(self.linear3(out))
        out = self.classifier(out)
        return out


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

Función de entrenamineto


In [7]:
# TRAIN NETWORK
def train_network(optimizer, epoch, criterion, net):
    train_loss, train_correct = 0, 0
    net.train()
    with tqdm(
        iter(train_dataloader), desc="Epoch " + str(epoch), unit="batch"
    ) as tepoch:
        for batch in tepoch:
            # Returned values of Dataset Class
            images = batch["img"].to(device)
            labels = batch["label"].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # Forward
            outputs = net(images)
            loss = criterion(outputs, labels)

            # Calculate gradients
            loss.backward()

            # Update parameters
            optimizer.step()

            # one hot -> labels
            labels = torch.argmax(labels, dim=1)
            pred = torch.argmax(outputs, dim=1)
            train_correct += pred.eq(labels).sum().item()

            # print statistics
            train_loss += loss.item()

    train_loss /= len(train_dataloader.dataset)
    return train_loss, train_correct

Función de test


In [8]:
# TEST NETWORK
def test_network(net, epoch, criterion):
    test_loss, test_correct = 0, 0
    net.eval()
    with torch.no_grad():
        with tqdm(
            iter(test_dataloader), desc="Test " + str(epoch), unit="batch"
        ) as tepoch:
            for batch in tepoch:
                images = batch["img"].to(device)
                labels = batch["label"].to(device)

                # Forward
                outputs = net(images)
                test_loss += criterion(outputs, labels)

                # one hot -> labels
                labels = torch.argmax(labels, dim=1)
                pred = torch.argmax(outputs, dim=1)

                test_correct += pred.eq(labels).sum().item()

        test_loss /= len(test_dataloader.dataset)
        test_accuracy = 100.0 * test_correct / len(test_dataloader.dataset)
    return test_loss, test_correct, test_accuracy

Entrenamiento con validación


In [ ]:
# Instantiating the network and printing its architecture
num_classes = 10
net = Net(num_classes)
print(net)
print("Params: ", count_parameters(net))
####################################################################
# Training settings
####################################################################

# Training hyperparameters
epochs = 15
criterion = nn.CrossEntropyLoss()
best_results = dict()
for lr in [0.1, 0.01, 0.001, 0.0001]:
    best_results[lr] = dict()
    for optimizer in [optim.AdamW, optim.SGD, optim.Adadelta, optim.Adam]:
        ####################################################################
        # Training
        ####################################################################

        # Load model in GPU, cada vez que hacemos un entrenamiento con un tipo de optimizador y lr tenemos que crear la red
        net = Net(num_classes)
        net.to(device)
        print("Params: ", count_parameters(net))
        # y el optimizador debe crearse despues de haber creado la red porque le pasamos los parámetros de la red que serán los que se encargue de optimizar
        if optimizer == optim.SGD:
            optimizer = optimizer(
                net.parameters(), lr=lr, weight_decay=1e-6, momentum=0.9
            )
        else:
            optimizer = optimizer(net.parameters(), lr=lr, weight_decay=1e-6)

        best_results[lr][optimizer.__class__.__name__] = {
            "best_accuracy": -1,
            "best_epoch": 0,
        }

        print("\n---- Start Training ----")
        print(f"Optimizer: {optimizer.__class__.__name__} - Learning Rate: {lr}")
        for epoch in range(epochs):
            train_loss, train_correct = train_network(optimizer, epoch, criterion, net)

            test_loss, test_correct, test_accuracy = test_network(net, epoch, criterion)

            print(
                "[Epoch {}] Train Loss: {:.6f} - Test Loss: {:.6f} - Train Accuracy: {:.2f}% - Test Accuracy: {:.2f}%".format(
                    epoch + 1,
                    train_loss,
                    test_loss,
                    100.0 * train_correct / len(train_dataloader.dataset),
                    test_accuracy,
                )
            )

            if (
                test_accuracy
                > best_results[lr][optimizer.__class__.__name__]["best_accuracy"]
            ):
                best_results[lr][optimizer.__class__.__name__][
                    "best_accuracy"
                ] = test_accuracy
                best_results[lr][optimizer.__class__.__name__]["best_epoch"] = epoch

                # Save best weights
                torch.save(net.state_dict(), "best_model.pt")

        print(
            "\nBEST TEST ACCURACY: ",
            best_results[lr][optimizer.__class__.__name__]["best_accuracy"],
            " in epoch ",
            best_results[lr][optimizer.__class__.__name__]["best_epoch"],
        )

####################################################################
# Results
####################################################################
print(best_results)

Net(
  (linear1): Linear(in_features=784, out_features=512, bias=True)
  (relu1): ReLU()
  (linear2): Linear(in_features=512, out_features=512, bias=True)
  (relu2): ReLU()
  (linear3): Linear(in_features=512, out_features=512, bias=True)
  (relu3): ReLU()
  (classifier): Linear(in_features=512, out_features=10, bias=True)
)
Params:  932362
Params:  932362

---- Start Training ----
Optimizer: AdamW - Learning Rate: 0.1


Test 0: 100%|██████████| 100/100 [00:02<00:00, 46.35batch/s]

[Epoch 1] Train Loss: 0.160598 - Test Loss: 0.020093 - Train Accuracy: 18.04% - Test Accuracy: 21.09%



Test 1: 100%|██████████| 100/100 [00:01<00:00, 52.05batch/s]

[Epoch 2] Train Loss: 0.019885 - Test Loss: 0.018422 - Train Accuracy: 20.72% - Test Accuracy: 25.13%


...


Test 12: 100%|██████████| 100/100 [00:01<00:00, 51.10batch/s]

[Epoch 13] Train Loss: 0.000155 - Test Loss: 0.000873 - Train Accuracy: 99.54% - Test Accuracy: 98.10%



Test 13: 100%|██████████| 100/100 [00:01<00:00, 51.86batch/s]

[Epoch 14] Train Loss: 0.000147 - Test Loss: 0.001040 - Train Accuracy: 99.54% - Test Accuracy: 97.80%



Test 14:  34%|███▍      | 34/100 [00:00<00:01, 51.96batch/s]

In [ ]:
print(best_results)

In [ ]:
# mostrar resultats del diccionari amb tabulate
for lr in best_results:
    for optimizer in best_results[lr]:
        best_results[lr][optimizer] = (
            round(best_results[lr][optimizer]["best_accuracy"], 2),
            best_results[lr][optimizer]["best_epoch"],
        )

headers = best_results[list(best_results.keys())[0]].keys()
values = [[name, *inner.values()] for name, inner in best_results.items()]

print(tabulate.tabulate(values, headers=headers))


best_accuracy = -1
best_lr = None
best_optimizer = None
for lr in best_results:
    for optimizer in best_results[lr]:
        if best_results[lr][optimizer][0] > best_accuracy:
            best_accuracy = best_results[lr][optimizer][0]
            best_lr = lr
            best_optimizer = optimizer

print(
    "\nBEST TEST ACCURACY: ",
    best_accuracy,
    " with lr: ",
    best_lr,
    " and optimizer: ",
    best_optimizer,
)
####################################################################
# Load best weights
####################################################################

# Load best weights, tenemos que tener una red creada con la misma arquitectura que la que hemos entrenado
net = Net(num_classes)
net.load_state_dict(torch.load("best_model.pt"))

_, _, test_accuracy = test_network(net, 0, criterion)
print("\nFinal best acc: ", test_accuracy)